In [ ]:
import pandas as pd
from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import Dropdown, Output, VBox, IntRangeSlider, Layout, Label

import os
%cd ..
from utils import read_json, build_option_expiries
from influxdb_wrapper import InfluxDBWrapper
%cd -

In [ ]:
config = read_json('../config.json')
bucket = 'btc_vol_surfaces'
wrapper = InfluxDBWrapper(config['database']['url'], config['database']['token'], config['database']['org'], db_timeout=30_000)
obs_time='2023-05-29T08:30:00Z'

In [ ]:
def show_vol_surface_for_obs_time(obs_time, field = 'mid_iv'):
    vol_surface = wrapper.get_vol_surface_for_obs_time(bucket, 'volatility', obs_time, field)

    # display from expiry T+1
    surface_display = vol_surface.iloc[1:]
    
    # reverse rows and columns
    surface_display = surface_display.iloc[::-1]
    surface_display = surface_display[vol_surface.columns[::-1]]

    fig = go.Figure(data=[go.Surface(x=surface_display.columns,
                                     y=surface_display.index,
                                     z=surface_display.values,
                                     contours = {
                                    "x": {"show": True, "start": 0, "end": 20, "size": 1, "color":"gray"},
                                    "y": {"show": True, "start": 0, "end": 10, "size": 1, "color":"gray"}},
                                    colorscale='rdylbu',
                                    reversescale=True)])


    fig.update_layout(
        title=f'Volatility Surface on {obs_time}',
        width=800,  
        height=600,  
        scene=dict(
            xaxis=dict(
                tickmode='array',  
                tickvals=list(range(len(surface_display.columns))),  
                ticktext=surface_display.columns, 
                title='Delta'
            ),
            yaxis=dict(
                tickmode='array',  
                tickvals=list(range(len(surface_display.index))),
                ticktext=surface_display.index, 
                title='Expiry',
                type = 'category'
            ),
            zaxis=dict(title=field)
        
        )
    )

    fig.show()
        
    return vol_surface

show_vol_surface_for_obs_time(obs_time=obs_time, field='mid_iv')


In [ ]:
def show_smiles(obs_time, field = 'mid_iv'):
    vol_surface = wrapper.get_vol_surface_for_obs_time(bucket, 'volatility', obs_time, field)
    

    maturities = vol_surface.index.to_list()
    traces = []
    # Iterate over each maturity and create a Scatter trace
    for idx, maturity in enumerate(maturities):
        vol_data = vol_surface.loc[[maturity]]

        if (idx == 3 or idx == 9):
            visible = True
        else:
            visible='legendonly'

        trace = go.Scatter(
            x=vol_data.columns,  # Deltas as x-axis
            y=vol_data.values[0],   # Implied vols as y-axis
            mode='lines+markers',
            name=maturity,
            visible=visible
        )

        traces.append(trace)

    min_vol = vol_surface.iloc[[3,9]].min().min() - 2
    max_vol = vol_surface.iloc[[3,9]].max().max() + 2

    layout = go.Layout(
        title=f'Smiles - {obs_time}',
        xaxis=dict(title='Delta'),
        yaxis=dict(title='Implied Volatility', range=[min_vol, max_vol]),
        height = 800,
        width = 1000
    )

    fig = go.Figure(data=traces, layout=layout)
    fig.show()

show_smiles(obs_time=obs_time, field='mid_iv')

In [ ]:
def show_vol_term_structure(obs_time, field = 'mid_iv'):
    vol_surface = wrapper.get_vol_surface_for_obs_time(bucket, 'volatility', obs_time, field)

    deltas = vol_surface.columns.to_list()
    traces = []
    
    initial_display = [2,4,9,14,16]
    
    for idx, delta in enumerate(deltas):
        vol_data = vol_surface[delta]

        if idx in initial_display:
            visible = True
        else:
            visible='legendonly'

        trace = go.Scatter(
            x=vol_data.index,  # Deltas as x-axis
            y=vol_data.values,   # Implied vols as y-axis
            mode='lines+markers',
            name=delta,
            visible=visible
        )

        traces.append(trace)

    min_vol = vol_surface.iloc[:,initial_display].min().min() - 5
    max_vol = vol_surface.iloc[:,initial_display].max().max() + 5

    layout = go.Layout(
        title=f'Vol term structure on {obs_time}',
        xaxis=dict(title='Expiry'),
        yaxis=dict(title='Implied Volatility', range=[min_vol, max_vol]),
        height = 800,
        width = 1200,
        updatemenus=[
            dict(
                type="buttons",
                # direction="right",
                active=0,
                buttons=[
                    dict(label="Expiries as Categories",
                         method="relayout",
                         args=[{"xaxis.type": "category"}]),
                    dict(label="Expiries as Dates",
                         method="relayout",
                         args=[{"xaxis.type": "date"}])
                ]
            )
        ]
    )

    fig = go.Figure(data=traces, layout=layout)
    fig.update_xaxes(type='category')

    # Show the graph
    fig.show()

show_vol_term_structure(obs_time=obs_time, field='mid_iv')

In [ ]:
def show_vol_history_for_delta_and_expiry(start, end, delta='ATM', expiry=None, field='mid_iv'):
    expiries = build_option_expiries(datetime.now())
    deltas = ['5P', '10P', '15P', '20P', '25P', '30P', '35P', '40P', '45P', 'ATM', '45C', '40C', '35C', '30C', '25C', '20C', '15C', '10C', '5C']

    if expiry is None:
        expiry = expiries[len(expiries)-3]

    delta_dropdown = Dropdown(options=deltas, value=delta, description='Delta:')
    expiry_dropdown = Dropdown(options=expiries, value=expiry, description='Expiry:')
    start_time = int(datetime.strptime(start, '%Y-%m-%dT%H:%M:%SZ').timestamp())
    end_time = int(datetime.strptime(end, '%Y-%m-%dT%H:%M:%SZ').timestamp())
    timerange_slider = IntRangeSlider(value=[start_time, end_time], min=start_time, max=end_time,
                                      layout=Layout(width='75%'), readout=False)
    time_range_label = Label(value=f"Time Range: {datetime.utcfromtimestamp(start_time).strftime('%Y-%m-%d')} to {datetime.utcfromtimestamp(end_time).strftime('%Y-%m-%d')}")
    output = Output()

    def update_graph(change):
        delta = delta_dropdown.value
        expiry = expiry_dropdown.value
        start_time, end_time = timerange_slider.value

        with output:
            output.clear_output()
            historical_volatilities = wrapper.get_historical_vol_for_delta_and_expiry(bucket, 'volatility',
                                                                                   datetime.utcfromtimestamp(start_time).strftime('%Y-%m-%dT%H:%M:%SZ'),
                                                                                   datetime.utcfromtimestamp(end_time).strftime('%Y-%m-%dT%H:%M:%SZ'),
                                                                                   delta, expiry, field)


            fig = go.Figure()
            fig.add_trace(go.Scatter(x=historical_volatilities['timestamp'], y=historical_volatilities[field],
                                     name=f"Delta {delta}, Expiry {expiry}"))

            fig.update_layout(
                title=f"Historical Volatility for Delta {delta} and Expiry {expiry}",
                xaxis_title="Timestamp",
                yaxis_title=field,
                height=600,
                width=1000
            )
            fig.show()

            # Update time range label
            time_range_label.value = f"Time Range: {datetime.utcfromtimestamp(start_time).strftime('%Y-%m-%dT%H:%M:%SZ')} to {datetime.utcfromtimestamp(end_time).strftime('%Y-%m-%dT%H:%M:%SZ')}"

    delta_dropdown.observe(update_graph, names='value')
    expiry_dropdown.observe(update_graph, names='value')
    timerange_slider.observe(update_graph, names='value')

    controls = VBox([delta_dropdown, expiry_dropdown, time_range_label, timerange_slider], layout=Layout(width='100%'))
    layout = VBox([output, controls])

    update_graph(None)  # Display initial graph

    display(layout)


show_vol_history_for_delta_and_expiry(start='2023-05-08T00:00:00Z', end=obs_time)
